# Quick start

The `fitgrid` data modeling workflow consists of 4 steps:

1. Prepare your dataset as a `pandas.DataFrame`
2. Load your dataset into fitgrid, which creates `Epochs`.
3. Run a model on the `Epochs`, which creates a `FitGrid`.
4. Query the `FitGrid` for the estimates and diagnostic information

The commands below are in Python and are executed in Jupyter, but any Python environment will work. We recommend Jupyter Lab, Jupyter Notebook, or IPython.

In [1]:
import fitgrid

## 1. Prepare your dataset

`fitgrid` assumes you are modeling "epochs", i.e., fixed-length segments of time-stamped data streams ("channels").

Prepare your dataset as a `pandas.DataFrame` layed out in a long rows x narrow columns format with the response and predictor variables as the named columns and the data values ("observations") in rows.

Besides the response and predictor columns, two additional index columns that give the epoch identifier and time stamp of each observation (see the examples below).


### Dataset Format

- column names can be chosen freely

- all epochs must exactly the same time stamps

- each epoch must have a unique identifier in the dataset, no duplicates are allowed


### 1.2 Example

Here is a toy dataset, `my_data`.

There are two predictor variable columns and two response variable columns.

There are 4 epochs:
    * each has a unique identifier, one of `1, 2, 3, 4`
    * all have the same 3 time stamps `0, 100, 200`


In [2]:
import numpy as np
import pandas as pd
n_timestamps = 3

categorical_factor = ["a", "b"]
n_each = 2  
n_epochs = len(categorical_factor) * n_each

my_data = pd.DataFrame(
    {
        'epochs': 1 + np.repeat(range(n_epochs), n_timestamps),
        'times': 100 * np.tile(range(n_timestamps), n_epochs),
        'predictor_x1': np.random.random(n_timestamps * n_epochs),
        'predictor_x2': np.tile(
            np.repeat(categorical_factor, n_timestamps), n_each
        ),
        'data_y1': np.random.random(n_timestamps * n_epochs),
        'data_y2': np.random.random(n_timestamps * n_epochs),
    }
)

# this index is required!
my_data.set_index(['epochs', 'times'], inplace=True)
my_data

predictor_x1 predictor_x2   data_y1   data_y2
epochs times                                               
1      0          0.484871            a  0.094379  0.812954
       100        0.970399            a  0.713902  0.447617
       200        0.425820            a  0.787637  0.503503
2      0          0.071507            b  0.030460  0.438876
       100        0.548807            b  0.836818  0.105475
       200        0.471725            b  0.261262  0.857289
3      0          0.893282            a  0.989750  0.547980
       100        0.797527            a  0.346154  0.102748
       200        0.800388            a  0.243537  0.121599
4      0          0.000189            b  0.219465  0.727331
       100        0.534327            b  0.597590  0.819707
       200        0.585838            b  0.223957  0.037704

## 2. Load your dataset into fitgrid

`fitgrid` can load your dataframe directly from the Python workspace or from a file.

Either way, you need to tell `fitgrid`:

1. which index column is the epoch identifier
2. which index column is the time identifier
3. which columns are the response variable(s) to model

### 2.1 Example: Load a dataset from the Python workspace

In [3]:
# feed the toy dataset to fitgrid
epochs_fg = fitgrid.epochs_from_dataframe(
    my_data,
    time='times',
    epoch_id='epochs',
    channels=['data_y1', 'data_y2']
)

Now you have a fitgrid epochs table

In [4]:
epochs_fg

### 2.2 Load a dataset from an HDF5 file

The file `example.h5` was saved with `pandas.to_hdf()`. 

It contains a toy dataset with 20 epochs and 100 timestamps that looks like this:

|   Epoch_idx  | Time  | continuous | categorical   | channel0   | channel1   |
|--------------|-------|------------|---------------|------------|------------|
|          0   |  0    | 0.439425   |     cat0      | -13.332813 |  24.074655 |
|          0   |  1    | 0.028030   |     cat0      | -16.005318 |  23.879106 |
|          0   |  2    | 0.484779   |     cat0      |  21.309482 |  13.479029 |
|          0   |  3    | 0.008352   |     cat0      | -39.315872 |  46.974077 |
|          0   |  4    | 0.597296   |     cat0      |  34.399671 |  -3.740801 |
|          .   |  .    |    .       |      .        |      .     |      .     |
|          .   |  .    |    .       |      .        |      .     |      .     |
|          .   |  .    |    .       |      .        |      .     |      .     |
|         19   | 95    | 0.611419   |     cat1      |   6.877276 |  -3.882082 |
|         19   | 96    | 0.728147   |     cat1      | -38.291487 |   1.024060 |
|         19   | 97    | 0.605416   |     cat1      |   4.123766 |  56.674669 |
|         19   | 98    | 0.199554   |     cat1      | -45.001713 | -18.420173 |
|         19   | 99    | 0.008011   |     cat1      | -30.901878 |  35.481089 |


To load this dataset directly, run:

In [5]:
epochs_fg = fitgrid.epochs_from_hdf(
    filename='example.h5',
    key=None,
    time='Time',
    epoch_id='Epoch_idx',
    channels=['channel0', 'channel1']
)

This also creates an epochs object

In [6]:
epochs_fg

### 2.3 Load a dataset from a feather format file

See `fitgrid` Reference


## 3. Run a model

Once the epochs are loaded, `fitgrid` fits a model (formula) at each time point and channel and captures the results in 2-D grid (time x channels).

Each cell in the grid has the model fit information for that time point and channel, such as estimated betas and diagnostic information like $R^2$ and Cook's $D$ in the case of linear regression.

Model formulas are the same ones used by `lme4:lmer` in `R` and `patsy` for `statsmodels` in `Python`

As of now, linear regression (via ``statsmodels``' ``ols``) and linear mixed
models (via ``lme4``'s ``lmer``) are available. 

Running a model on the epochs creates a `FitGrid` object, 

### 3.1 Ordinary least squares

To run linear regression on the epochs, use the `lm` function. This calls `statsmodels.ols` under the hood.

In [7]:
lm_grid = fitgrid.lm(
    epochs_fg,
    RHS='continuous + categorical'
)

100%|██████████| 100/100 [00:01<00:00, 54.45it/s]


`fitgrid.lm` runs linear regression for each channel, with a single channel
data as the left hand side, and the right hand side given by the Patsy/R style
formula passed in using the `RHS` parameter:

    channel0 ~ continuous + categorical
    channel1 ~ continuous + categorical
    ...
    channel31 ~ continuous + categorical

If you want to model only a specific subset of channels, pass the list of channels to the `LHS` parameter.

```python
lm_grid = fitgrid.lm(
    epochs_fg,
    LHS=['channel1', 'channel3', 'channel5'],
    RHS='continuous + categorical'
)
```

### 3.2 linear mixed effects

Similarly, to model linear mixed effects use the `lmer` function. This calls `lme4::lmer()` under the hood.

In [8]:
%%time

lmer_grid = fitgrid.lmer(
    epochs_fg,
    RHS='continuous + (continuous | categorical)'
)

  0%|          | 0/100 [00:00<?, ?it/s]/home/turbach/.conda/envs/fitgrid_dev/lib/python3.6/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)
100%|██████████| 100/100 [02:16<00:00,  1.27s/it]

CPU times: user 2min 12s, sys: 3.87 s, total: 2min 16s
Wall time: 2min 16s


### 3.3 Multicore parallel processing

With lmer especially, it may be useful to run your model with multiple
processes to speed it up.

This can be achieved by setting ``parallel`` to
``True`` and ``n_cores`` to the desired value (defaults to 4) as follows.

The performance benefits depend on your hardware. A laptop usually has 4 cores, a good workstation may have 8, a high performance server may have dozens. If working on a shared system it is rude (and unproductive) to hog cores.

In [9]:
%%time

lmer_grid = fitgrid.lmer(
    epochs_fg,
    RHS='continuous + (continuous | categorical)', 
    parallel=True,
    n_cores=4)

100%|██████████| 100/100 [00:00<00:00, 542.21it/s]
/home/turbach/.conda/envs/fitgrid_dev/lib/python3.6/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)
/home/turbach/.conda/envs/fitgrid_dev/lib/python3.6/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)
/home/turbach/.conda/envs/fitgrid_dev/lib/python3.6/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(it

CPU times: user 675 ms, sys: 128 ms, total: 803 ms
Wall time: 38.7 s


## 4. Working with the grid


``FitGrid`` objects, like `lm_grid` or `lmer_grid` above, can be queried for attributes just like a
``fit`` object returned by ``statsmodels`` or ``lmer`` (see Overview Doing Statistics in Python for more
background), for example:

**Hint**: If you are using an interactive environment like Jupyter Notebook or IPython,
  you can use tab completion to see what attributes are available:

```python
# type 'lm_grid.' and press Tab
lm_grid.<TAB>
```

### 4.1 Examples

#### Beta estimates

In [10]:
betas = lm_grid.params
betas.head(6)

channel0   channel1
Time                                          
0    Intercept            18.050893  32.704467
     categorical[T.cat1] -17.577082  11.820177
     continuous          -14.427532 -55.349303
1    Intercept            15.078748  -2.270738
     categorical[T.cat1] -19.521091  -1.673252
     continuous          -16.362195  -3.327804

####  Adjusted $R^2$

In [11]:
rsquared_adj = lm_grid.rsquared_adj
rsquared_adj.head(6)

,channel0,channel1
Time,,
0,0.006194,0.260604
1,0.072003,-0.115794
2,0.052775,-0.054846
3,-0.076032,0.233450
4,-0.004449,0.038667
5,-0.115338,0.025115


#### Cook's distance

In [12]:
influence = lm_grid.get_influence()
cooks_distance = influence.cooks_distance
cooks_distance.head()

channel0  channel1
Time   Epoch_idx                    
0    0 0          0.034113  0.015758
       1          0.060702  0.015379
       2          0.054559  0.034056
       3          0.008012  0.044054
       4          0.134372  0.279701

### 4.2 Queries return a `DataFrame` or another `FitGrid`

Calling an attribute of a `FitGrid` objects returns either a pandas `DataFrame` of the
appropriate shape or another `FitGrid` object:

In [13]:
# this is a dataframe
lm_grid.params.head()

channel0   channel1
Time                                          
0    Intercept            18.050893  32.704467
     categorical[T.cat1] -17.577082  11.820177
     continuous          -14.427532 -55.349303
1    Intercept            15.078748  -2.270738
     categorical[T.cat1] -19.521091  -1.673252

In [14]:
# this is a FitGrid
lm_grid.get_influence()

100 by 2 FitGrid of type <class 'statsmodels.stats.outliers_influence.OLSInfluence'>.

If a dataframe is returned, it is always presented in long form with the same
indices and columns on the outer side as a single epoch: channels as columns
and time as indices.

### 4.3 Slicing the grid

In addition, slicing on a `FitGrid` can be performed to produce a smaller grid
of the shape you want. Suppose you want to only look at a certain channel
within a given time interval. You can slice as follows:

In [15]:
smaller_grid = lm_grid[25:75, 'channel0']
smaller_grid

51 by 1 LMFitGrid of type <class 'statsmodels.regression.linear_model.RegressionResultsWrapper'>.

Or multiple channels:

In [16]:
smaller_grid = lm_grid[25:75, ['channel0', 'channel1']]
smaller_grid

51 by 2 LMFitGrid of type <class 'statsmodels.regression.linear_model.RegressionResultsWrapper'>.

To include all timepoints or all channels, use a colon:

In [17]:
# all channels within certain timeframe
lm_grid[25:75, :]

51 by 2 LMFitGrid of type <class 'statsmodels.regression.linear_model.RegressionResultsWrapper'>.

In [18]:
# all timepoints, two channels
lm_grid[:, ['channel0', 'channel1']]

100 by 2 LMFitGrid of type <class 'statsmodels.regression.linear_model.RegressionResultsWrapper'>.